In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
flt = pd.read_csv("FrenchTable.csv")

In [22]:
def simulate_life(start_age, life_table):
    age = start_age
    dead = False
    while not dead:
        died_during_year = np.random.binomial(n = 1, p = flt[flt['Age'] == age]['qx'])[0]
        if died_during_year == 1:
            dead = True
        else:
            age += 1
    
    return age

In [46]:
def whole_life_insurance_premium(start_age, life_table, interest_rate, principal = 1):
    return principal * (1 - (1 - 1 / (1 + interest_rate)) * life_annuity_premium(start_age, life_table, interest_rate, "due"))

In [53]:
def life_annuity_premium(start_age, life_table, interest_rate, annuity_type = "due", payout = 1):
    x = start_age
    
    if annuity_type == "due":
        t = 0
    elif annuity_type == "immediate":
        t = 1
    else:
        raise ValueError("Incorrect value for annuity_type. Needs to be either 'due' or 'immediate'")
     
    value = 0
    
    while start_age + t < max(life_table['Age']):
        value += v(interest_rate, t) * tpx(start_age, t, life_table)
        t += 1
        
    return value * payout

In [48]:
def tpx(x, t, life_table):
    
    tpx = 1
    
    for i in range(t):
        tpx *= (1 - life_table[life_table['Age'] == x + i]['qx'].item())
        
    return tpx

In [56]:
def simulate_life_insurance(start_age, life_table, interest_rate, principal):
    age = start_age
    dead = False
    accumulated_premiums = []
    
    while not dead:
        accumulated_premiums.append(whole_life_insurance_premium(start_age, life_table, interest_rate, principal))
        died_during_year = np.random.binomial(n = 1, p = flt[flt['Age'] == age]['qx'])[0]
        if died_during_year == 1:
            dead = True
        else:
            age += 1
    
    for t in range(len(accumulated_premiums)):
        accumulated_premiums[t] *= v(interest_rate, t)

    print("The policyholder died at age %d." % age)
    print("Present value of premiums: %.2lf" % sum(accumulated_premiums))
    
    '''
    if sum(accumulated_premiums) > principal:
        print("You made a gain of %.2lf" % sum(accumulated_premiums) - principal)
    elif sum(accumulated_premiums) == principal:
        print("You have not made a gain or loss.")
    else:
        print("You made a loss of %.2lf" % principal - sum(accumulated_premiums))
    
    '''
        
    return 

In [50]:
def tqx(x, t, life_table):
    return 1 - tpx(x, t, life_table)

In [52]:
def v(i, t):
    return 1 / (1 + i) ** t

In [57]:
simulate_life_insurance(30, flt, 0.05, 100000)

The policyholder died at age 67.
Present value of premiums: 256535.66
